# Massachusetts tax and benefit system

This notebook shows how the state and federal tax and benefit system affects Massachusetts residents holistically.

## Examples

Consider a set of Massachusetts family types, each with $1,000 monthly rent and $50 monthly broadband costs, and varying in terms of marital status and number of children.
Their net income---after state and federal taxes and benefits modeled by OpenFisca US---is shown in the graph below.

*The cliff is due to Massachusetts' emergency SNAP allotment, which entitles SNAP-eligible households to the maximum benefit for their household size; this also affects other benefits through categorical eligibility.*

In [5]:
from openfisca_us import IndividualSim
import pandas as pd
import plotly.express as px


def make_tax(adults, children):
    sim = IndividualSim(year=2022)
    sim.add_person(name="head", age=25)
    members = ["head"]
    if adults == 2:
        sim.add_person(name="spouse")
        members += ["spouse"]
    for i in range(children):
        child = "child{}".format(i)
        sim.add_person(name=child, age=6)
        members += [child]
    sim.add_tax_unit(name="tax_unit", members=members)
    # $1,000 monthly rent, $50 monthly broadband.
    sim.add_spm_unit(name="spm_unit", members=members, rent=12_000, broadband_cost=600)
    sim.add_household(name="household", members=members, state_code="MA")
    sim.vary("employment_income", max=100_000, step=100)
    return pd.DataFrame(
        dict(
            employment_income=sim.calc("employment_income")[0],
            spm_unit_net_income=sim.calc("spm_unit_net_income")[0].round(),
            mtr=1-sim.deriv(
                "spm_unit_net_income", "employment_income", wrt_target="head"
            ),
            adults=adults,
            children=children,
        )
    )


# Make a table of state taxes for different numbers of adults and children.
l = []
for adults in range(1, 3):
    for children in range(0, 4):
        l.append(make_tax(adults, children))

df = pd.concat(l)

LABELS = dict(
    employment_income="Employment income",
    spm_unit_net_income="Net income",
    mtr="Marginal tax rate",
    adults="Adults",
    children="Children",
)

fig = px.line(
    df,
    "employment_income",
    "spm_unit_net_income",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Net income for a Massachusetts household",
)
fig.update_layout(xaxis_tickformat="$,", yaxis_tickformat="$,")
fig.show()

Total marginal tax rates range from -66% to 52%.

In [6]:
fig = px.line(
    df,
    "employment_income",
    "mtr",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Marginal tax rate for a Massachusetts household",
)
fig.update_layout(xaxis_tickformat="$,", yaxis_tickformat=".1%", yaxis_range=[-1, 1])
fig.show()